# Extract Signs from ORACC JSON
The code in this notebook will parse [ORACC](http://oracc.org) `JSON` files to extract signs from the Sumerian texts of one or more projects. 

In [1]:
import pandas as pd
import zipfile
import json
import tqdm
import requests
import errno
import os
import pickle
import re

## 0 Create Directories, if Necessary
The two directories needed for this script are `jsonzip` and `output`. If they do not exist they are created, else: do nothing.

For the code, see [Stack Overflow](http://stackoverflow.com/questions/18973418/os-mkdirpath-returns-oserror-when-directory-does-not-exist).

In [2]:
directories = ['jsonzip', 'output']
for d in directories:
    try:
        os.mkdir(d)
    except OSError as exc:
        if exc.errno !=errno.EEXIST:
            raise
        pass

## 1.1 Input Project Names
Provide a list of one or more project names, separated by commas. Note that subprojects must be listed separately, they are not included in the main project. For instance:

`epsd2/admin/ed3a, epsd2/admin/ed3b, epsd2/admin/ebla, epsd2/admin/oakk, epsd2/admin/lagash2, epsd2/admin/u3adm, epsd2/admin/u3let, epsd2/admin/u3leg, epsd2/admin/oldbab, epsd2/literary, epsd2/emesal, epsd2/earlylit, epsd2/royal, epsd2/praxis, dcclt, obmc, ckst, blms`

In [3]:
projects = input('Project(s): ').lower()

Project(s): epsd2/admin/ed3a, epsd2/admin/ed3b, epsd2/admin/ebla, epsd2/admin/oakk, epsd2/admin/lagash2, epsd2/admin/u3adm, epsd2/admin/u3let, epsd2/admin/u3leg, epsd2/admin/oldbab, epsd2/literary, epsd2/emesal, epsd2/earlylit, epsd2/royal, epsd2/praxis, dcclt, obmc, ckst, blms


## 1.2 Split the List of Projects
Split the list of projects and create a list of project names.

In [4]:
p = projects.split(',')               # split at each comma and make a list called `p`
p = [x.strip() for x in p]        # strip spaces left and right of each entry in `p`

## 1.3 Download the ZIP files
For each project in the list download all the `json` files from `http://build-oracc.museum.upenn.edu/json/`. The file is called `PROJECT.zip` (for instance: `dcclt.zip`). For subprojects the file is called `PROJECT-SUBPROJECT.zip` (for instance `cams-gkab.zip`). 

For larger projects (such as [DCCLT](http://oracc.org/dcclt)) the `zip` file may be 25Mb or more. Downloading may take some time and it may be necessary to chunk the downloading process. The `iter_content()` function in the `requests` library takes care of that.

If you have downloaded the files by hand (and put them in the `jsonzip` directory) you may skip this cell and jump directly to section [2.1 The Parsejson() function](#head21).

In [ ]:
CHUNK = 16 * 1024
for project in tqdm.tqdm(p):
    project = project.replace('/', '-')
    url = "http://build-oracc.museum.upenn.edu/json/" + project + ".zip"
    file = 'jsonzip/' + project + '.zip'
    r = requests.get(url)
    if r.status_code == 200:
        print("Downloading " + url + " saving as " + file)
        with open(file, 'wb') as f:
            for c in r.iter_content(chunk_size=CHUNK):
                f.write(c)
    else:
        print(url + " does not exist.")

## <a name="head21"></a>2.1 The `parsejson()` function
The `parsejson()` function will "dig into" the `json` file (transformed into a dictionary) until it finds the relevant data. The `json` file consists of a hierarchy of `cdl` nodes; only the lowest nodes contain lemmatization data. The function goes down this hierarchy by calling itself when another `cdl` node is encountered. For nore information about the data hierarchy in the [ORACC](http://oracc.org) `json` files, see [ORACC Open Data](http://oracc.museum.upenn.edu/doc/opendata/index.html).

The argument of the `parsejson()` function is a `JSON` object, a dictionary that initially contains the entire contents of the original JSON file. The code takes the key `cdl` which itself contains an array (a list) of `JSON` objects. Iterating through these objects, if an object contains another `cdl` node, the function calls itself with this object as first argument. This way the function digs deeper and deeper into the `JSON` tree, until it does not encounter a `cdl` key anymore. Here we are at the level of individual words. The code checks for a key `f`, if it exists the signs are in the node `gdl` within the `f` node. 

In [5]:
def parsejson_signs(text):
    for JSONobject in text["cdl"]:
        field = ''
        if "cdl" in JSONobject: 
            parsejson_signs(JSONobject)
        if "type" in JSONobject and JSONobject["type"] == "field-start":
            field = JSONobject["subtype"]
        if "f" in JSONobject and not field in ['sg', 'pr']: # skip the fields "sign" and "pronunciation"
                                # in lexical texts
            if JSONobject["f"]["lang"][:3] == "sux": #only Sumerian and Emesal
                f = JSONobject["f"]["form"]
                all_.append(f)
    all_.append("\nEndofDoc")
    return

## 2.2 Call the `parsejson()` function for every `JSON` file
The code in this cell will iterate through the list of projects entered above (1.1). For each project the `JSON` zip file is located in the directory `jsonzip`, named PROJECT.zip. 

Each of these files is extracted from the `zip` file and read with the command `json.loads()`, which reads the json data and transforms it into a Python dictionary (a sequence of keys and values).

This dictionary, which is called `text` is now sent to the `parsejson()` function. The function adds signs to the `sign_l` list.

In [7]:
all_ = []
for project in p:
    file = "jsonzip/" + project.replace("/", "-") + ".zip"
    try:
        z = zipfile.ZipFile(file)       # create a Zipfile object
    except:
        print(file + " does not exist or is not a proper ZIP file")
        continue
    files = z.namelist()     # list of all the files in the ZIP
    files = [name for name in files if "corpusjson" in name and name[-5:] == '.json']                                                                                                  #that holds all the P, Q, and X numbers.
    for filename in tqdm.tqdm(files):                            #iterate over the file names
        id_text = project + filename[-13:-5] # id_text is, for instance, blms/P414332
        try:
            text = z.read(filename).decode('utf-8')         #read and decode the json file of one particular text
        #    print(filename)
            data_json = json.loads(text)                # make it into a json object (essentially a dictionary)
            parsejson_signs(data_json)
        except:
            print(id_text + ' is not available or not complete')

  0%|          | 0/4944 [00:00<?, ?it/s]

epsd2/admin/ebla/P241764 is not available or not complete
epsd2/admin/ebla/P315437 is not available or not complete
epsd2/admin/ebla/P315459 is not available or not complete


  0%|          | 284/71496 [00:00<01:18, 905.76it/s]

epsd2/admin/u3adm/P511905 is not available or not complete
epsd2/admin/u3adm/P511471 is not available or not complete


  1%|          | 619/71496 [00:00<01:08, 1032.85it/s]

...

100%|██████████| 394/394 [00:05<00:00, 71.45it/s]
0it [00:00, ?it/s]
 68%|██████▊   | 164/242 [00:00<00:00, 309.08it/s]

epsd2/praxis/P222388 is not available or not complete
epsd2/praxis/P342998 is not available or not complete
epsd2/praxis/P345800 is not available or not complete


  0%|          | 0/4290 [00:00<?, ?it/s]

epsd2/praxis/P307377 is not available or not complete


100%|██████████| 397/397 [00:01<00:00, 237.60it/s]


## 3 Data Structuring
### 3.1 Transform the Data into a DataFrame


In [8]:
words_l = []
word = []
separators = ['{', '}', '-']
separators2 = ['.', '+', '|']
operators = ['&', '%', '@', '×']
for e in tqdm.tqdm(all_):
    word = []
    for s in separators: # first split word into signs
        e = e.replace(s, ' ').strip()
    s_l = e.split()
    for sign in s_l:
        if sign[-1] == ')': # qualified sign - get only the qualifier
            stack = []  # |GIŠ×(GIŠ%GIŠ)|(LAK277) becomes LAK277
            ind = {}    # LAK277(|GIŠ×(GIŠ%GIŠ)|) becomes |GIŠ×(GIŠ%GIŠ)|
            for i, c in reversed(list(enumerate(sign))):
                if c == ')':
                    stack.append(i)
                if c == '(':
                    ind[stack.pop()] = i
            start = ind[len(sign)-1]
            t = sign[start+1:-1]
            if not t.islower():  # leave 1(diš) etc. alone.
                sign = t
        if '|' in sign:  # separate |DU.DU| and |DU+DU| into its components but not |DU&DU|
                        # and also not |DU.DU&DU|
            flag = False
            for o in operators:
                if o in e:
                    flag = True
            if not flag:
                for s in separators2:
                    sign = sign.replace(s, ' ').strip() 
                    sign_l = sign.split()
                word.extend(sign_l)
            continue
        elif "+" in sign:  # + as marker of gloss
            sign = sign.replace('+', ' ').strip()
            sign_l = sign.split()
            word.extend(sign_l)
            continue
        word.append(sign)
    words_l.append(word)           

100%|██████████| 4849121/4849121 [00:23<00:00, 205842.71it/s]


In [9]:
with open("output/ogsl.p", "rb") as f:
    o = pickle.load(f)

In [10]:
val = list(o["value"])
utf = list(o["utf8"])
names = list(o["name"])

In [11]:
d = dict(zip(names, utf))
d2 = dict(zip(val,names))

In [12]:
names_l = []
utf8_l = []
for w in tqdm.tqdm(words_l):
    seq = [d2[s.lower()] if s.lower() in d2 else s for s in w]
    names_l.append(seq)
    utf8 = [d[n] if n in d else n for n in seq]
    utf8_l.append(''.join(utf8))

100%|██████████| 4849121/4849121 [00:16<00:00, 300743.21it/s]


In [13]:
df = pd.DataFrame({"transliteration":all_, "words":words_l, "names":names_l, "utf-8":utf8_l})
df

,names,transliteration,utf-8,words
0,[DIŠ],1(barig@c),𒁹,[1(barig@c)]
1,[ŠE],še,𒊺,[še]
2,"[BA, LUL]",ba-lul,𒁀𒈜,"[ba, lul]"
3,[NAGAR],nagar,𒉄,[nagar]
4,[DIŠ],1(barig@c),𒁹,[1(barig@c)]
5,"[GAR, |U.GUD|]",nig₂-du₇,𒃻𒌌,"[nig₂, du₇]"
6,[|NINDA₂×NE|],ag₂,𒉘,[ag₂]
7,"[|HI×AŠ₂|, SAG, EŠ₂, MAH]",hur-sag-še₃-mah,𒄯𒊕𒂠𒈤,"[hur, sag, še₃, mah]"
8,"[SAG, DUN₃]",sa₁₂-du₅,𒊕𒂅,"[sa₁₂, du₅]"
9,[EndofDoc],\nEndofDoc,EndofDoc,[EndofDoc]


In [14]:
with open("output/sux.p", "wb") as w:
    pickle.dump(df, w)

In [15]:
sux_text = ' '.join(df['utf-8']).strip()
sux_text = re.sub(r'(EndofDoc )+', '\n', sux_text)

In [16]:
with open("output/sux.txt", 'w', encoding="utf-8") as w:
    w.write(sux_text)

In [17]:
print(sux_text[:10000])

𒁹 𒊺 𒁀𒈜 𒉄 𒁹 𒃻𒌌 𒉘 𒄯𒊕𒂠𒈤 𒊕𒂅 
2(iku@c) 𒃷 𒄯𒌅𒀭𒋢𒆳𒊒 𒉡𒊬 2(iku@c) 𒈨𒍣𒉺𒌓𒁺 2(iku@c) 1/2(iku@c) 𒄯𒌅𒀭𒋢𒆳𒊒 𒊮𒄘𒁀 4(iku@c) 1/2(iku@c) 𒀾𒄷𒄭 3(iku@c) 𒇯𒇯 𒍑𒆪 2(iku@c) 𒀀𒇻𒇻 𒄘𒋗𒃮 1(iku@c) 𒀭𒂗𒍪𒄷𒄭 2(iku@c) 𒇯𒈤 2(iku@c) 1/2(iku@c) 𒂍𒈾 𒈯 1(iku@c) 1/4(iku@c) 𒀫𒉣𒇬 1/4(iku@c) 𒌉𒋖 2(iku@c) 1/2(iku@c) 𒌨𒌉𒍣 𒀿 2(iku@c) 𒌑𒅗𒅗 𒍑𒆪 4(iku@c) 𒀭𒉡𒈨 𒅖 2(iku@c) 𒀫𒆬 1(iku@c) 𒈝𒈠 5(iku@c) 𒀭𒋢𒆳𒊒𒌨𒊕 𒉄 𒑘 1(iku@c) 1/2(iku@c) 𒉽 X 1(iku@c) X 𒊕LAK244 𒀭𒉡𒈨 𒊺 𒉘 
𒌋𒌋 𒇲 𒀸 𒄨 𒀭𒉡𒈨 𒌋 𒐂 𒌨𒀭𒋢𒆳𒊒 𒐕 𒐀 𒊩𒃾 𒐕 𒌋 𒐁 𒌨𒀭𒋢𒆳𒊒 𒐕 𒉽𒄑𒉈𒀭𒈪𒄷 𒐕 𒀸 𒈩𒇻𒇻 𒐕 𒌋 𒌉𒀭𒈪𒄷 𒐕 𒌋 𒐄 𒀜𒁕 𒐕 𒐁 𒁲𒌓 𒌋𒌋 𒐃 𒉄 𒌋𒌋 𒌣 𒌋 𒀸 𒀿 𒐁 𒉡𒊬 𒌋𒌋 𒐁 𒉣𒈨 𒌋𒌋 𒐅 𒃲𒂇 𒌍 𒐃 𒊕𒂅 𒌋𒌋 𒐂 𒋖𒁕𒉡𒌇 𒌋 𒐃 𒂍𒁍𒉘 𒌋 𒇲 𒀸 𒊩𒌆𒌨𒊕 𒌋𒌋 𒀫𒈹𒍝 𒌋𒌋 𒇲 𒐀 𒀫𒉣𒇬 𒌋 𒐀 𒀫𒉆𒉪 𒌋 𒐀 𒌨𒌉𒍣 𒐕 𒍠𒄩 𒋗𒆸 𒐞 𒐘 𒌋𒌋 𒇲 𒐀 𒄨 
𒁹 𒋀𒌉 𒐌 𒈩𒇽𒉡𒂠 𒁹 𒉽𒄑𒉈𒀉𒉡𒊨 𒐌 𒁈𒃶𒉡𒁲 𒁹 𒇯𒇯 𒁹 𒋛𒆕 𒁹 𒂗𒁽 𒐋 𒉺𒇻 X 𒄑 X 𒁁 
𒇉𒁈𒋛 𒈗𒉿 
𒀸 𒈗𒂗𒅍 𒉺𒄛 𒀸 𒁾𒊬 𒀸 𒁈𒃶𒉡𒁲 𒀸 𒃻𒌷𒀀𒍣 𒀸 𒃻𒈝𒈠 
𒐂 𒀲 𒀫𒈹𒍝 𒀀𒆠𒃲 𒐂 𒇯𒇯 𒋚 𒐂 𒈩𒆠𒈾 𒀳 𒐂 𒊩𒌆𒄘𒃲 𒍑𒆪 𒐂 𒉺𒄑𒉽𒉈𒂵 𒈾𒃰 𒐂 𒀾𒄷𒄭 𒈨 𒀭𒉈 𒐂 𒍪𒀊𒄷𒄭 𒀭𒉺𒇻 𒐂 𒌨𒌉𒍣 𒋛𒆕 𒐂 𒂍𒊊𒃲 𒉌𒌇 𒐂 𒀀𒉌𒉌 𒀭𒊩𒌆𒉺 𒃲𒌺 𒐁 𒂍𒈾𒇻 𒀭𒉡𒈲𒁕 𒐁 𒅋𒈲𒉡𒈨𒊒 𒐀 𒅗𒂵𒉌 𒁔𒈠 𒐀 𒀫𒆬 𒆥𒉪 𒐀 X 𒐀 𒈗𒃶𒅅 𒅁 𒐀 𒃻𒋜 𒐀 𒉆𒈤 𒂍𒉺𒌓𒁺 𒐀 𒌨𒌉𒍣 𒐀 𒆥𒉪 𒍑𒆪 𒐀 X 𒀝𒌓 XX 𒀭𒉺𒇻 𒐀 𒊍𒕆 𒉡𒌉 𒐀 𒄑𒉽𒉈𒄭 𒉺 𒐀 𒀭𒂗𒆤𒌺𒀀 𒈛 𒀭𒂠𒄘 𒌋𒌋 𒇲 3/4(aš@c) 𒀲𒀳 𒋚 𒌑𒂈 
X 𒐀 𒌉𒌉 𒁲𒌓 𒉺 𒇽𒀜 X 𒀭𒋢𒆳𒊒𒌨𒊕 𒉺 𒌋𒌋 𒐃 𒌨𒂍𒃲 𒉺 𒌋𒌋 𒈜 𒀝 𒌋 

In [ ]:
print("udu \nmaš")